In [ ]:
# Symmetrize half cells

In [ ]:
import numpy as np
import k3d
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
import traits.api as tr

In [ ]:
class WBScannedHalfCell(WBScannedCell):
    F_Cf = tr.Array(dtype=np.uint32,
                 value=[[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 7,  0],
       [5, 2]])
    isc_N_L = tr.List([[0, 7, 6], [1, 8, 5], [3, 2, 4, 8], [1, 2], [4, 5]])
    icrease_lines_N_Li = tr.Array(dtype=np.int_, value=[[0,1], [1,3], [2, 3], [4, 2], [1, 4], [2, 1]])
    L_x = tr.Float(100)


    def _get_O_basis_ab(self):
        """Derive the basis of the waterbomb cell.
        """
        Or = 2
        Fu, Fl = 3,4
        vec_Ox_a = self.isc_vectors_Li[3]
        nvec_Ox_a = -vec_Ox_a / np.linalg.norm(vec_Ox_a)
        O_a = self.icrease_nodes_X_Na[Or] + vec_Ox_a * self.L_x
        _vec_Oz_a = (self.normals_Fa[Fu] + self.normals_Fa[Fl]) / 2
        _nvec_Oz_a = _vec_Oz_a / np.linalg.norm(_vec_Oz_a)
        nvec_Oy_a = np.cross(_nvec_Oz_a, nvec_Ox_a)
        nvec_Oz_a = np.cross(nvec_Ox_a, nvec_Oy_a)
        O_basis_ab = np.array([nvec_Ox_a, nvec_Oy_a, nvec_Oz_a], dtype=np.float32)
        return O_a, O_basis_ab
    
    def _get_O_crease_nodes_X_Na(self):
        length_mid = self.L_x + (self.O_icrease_nodes_X_Na[[2,3,4],0] - 
                                self.O_icrease_nodes_X_Na[2,0])
        mid_Ca = self.O_icrease_nodes_X_Na[[2,3,4]]
        vec_mid_Ca = -self.O_isc_vectors_Li[[3,3,3]]
        mid_node_X_Ca = mid_Ca + vec_mid_Ca * length_mid

        length_valley = np.average(self.lengths_icrease_lines_L[[2,3]])
        valley_Ca = self.O_icrease_nodes_X_Na[[1,1]]
        vec_valley_Ca = self.O_isc_vectors_Li[[0,6]]
        valley_node_X_Ca = valley_Ca + vec_valley_Ca * length_valley

        length_mountain = 500 # np.average(self.lengths_icrease_lines_L[[13,14]])
        mountain_Ca = self.O_icrease_nodes_X_Na[[0]]
        vec_mountain_Ca = self.O_isc_vectors_Li[[7]]
        mountain_node_X_Ca = mountain_Ca + vec_mountain_Ca * length_mountain / 2
        valley_node_X_Ca, mountain_node_X_Ca

        corner_node_X_Ca = np.copy(valley_node_X_Ca)
        corner_node_X_Ca[:,0] = mountain_node_X_Ca[[0,0],0]
        O_bcrease_nodes_X_Ca = np.vstack([valley_node_X_Ca, mountain_node_X_Ca, corner_node_X_Ca, mid_node_X_Ca])
        O_crease_nodes_C_Ca = np.vstack([self.O_icrease_nodes_X_Na, O_bcrease_nodes_X_Ca])
        if self.O_flip < 0:
            T_ab = np.array([
                [np.cos(np.pi), np.sin(np.pi), 0 ],
                [-np.sin(np.pi), np.cos(np.pi), 0],
                [0, 0, 1]], dtype=np.float_)
            O_crease_nodes_C_Ca = np.einsum('ab,...a->...b', T_ab, O_crease_nodes_C_Ca)[self.O_crease_nodes_flip]

        return O_crease_nodes_C_Ca


In [ ]:
cell = WBScannedHalfCell(file_path="WB313R_facets_points.obj",
                     rotate_system=[[1,2],[np.pi/2, np.pi]])

In [ ]:
cell.F_Cf

In [ ]:
plot = k3d.plot(name='Nodes')
cell.plot_groups_of_points(plot, cell.wb_scan_X_Fia)
plot.display()

In [ ]:
plot = k3d.plot(name='Vectors')
cell.plot_groups_of_points(plot, cell.wb_scan_X_Fia)
cell.plot_planes(plot)
plot.display()

In [ ]:
plot = k3d.plot(name='Vectors')
cell.plot_groups_of_points(plot, cell.wb_scan_X_Fia)
cell.plot_points(plot, cell.centroids_Fa, point_size=20, 
                     color=0x000000, plot_numbers=False)
cell.plot_points(plot, cell.isc_points_Li, point_size=20, 
                     color=0x0000ff, plot_numbers=False)
cell.plot_intersection_lines(plot)
plot.display()

In [ ]:
plot = k3d.plot(name='Vectors')
cell.plot_points(plot, cell.icrease_nodes_X_Na, point_size=5, 
                     color=0x0000ff, plot_numbers=True)
cell.plot_intersection_lines(plot)
plot.display()

In [ ]:
plot = k3d.plot(name='Vectors')
cell.plot_icrease_nodes(plot, node_numbers=True)
cell.plot_icrease_lines(plot, line_numbers=True)
cell.plot_O_basis(plot)
plot.display()

In [ ]:
plot = k3d.plot(name='Vectors')
cell.plot_O_crease_nodes(plot, node_numbers=True)
#cell.plot_O_crease_lines(plot, line_numbers=True)
plot.display()